In [1]:
import pandas as pd
import re
import glob
import datetime
import numpy as np

In [2]:
#Import groups of customer data
appended_data = []
for file in glob.glob('Cust*'):
    data = pd.read_csv(file)
    appended_data.append(data)
cust_df = pd.concat(appended_data)
cust_df.head()

,owner_no,First Order Date,First Contribution Date,postal_code,state_desc,geo_area_desc,OP Prelim Capacity,LTV Tkt Value,Lifetime Giving
0,2280536,2015-01-01 00:00:00,NaN,8807,New Jersey,2-Greater Philadelphia(70 mi.),NaN,258.0,NaN
1,2280550,2015-01-01 00:00:00,NaN,18974,Pennsylvania,1-Philadelphia City (20 mi.),NaN,153.0,NaN
2,2280469,2015-01-01 00:00:00,NaN,190068506,Pennsylvania,1-Philadelphia City (20 mi.),3,210.0,NaN
3,2269456,2015-01-01 00:00:00,NaN,19403,Pennsylvania,1-Philadelphia City (20 mi.),4,281.0,NaN
4,2280674,2015-01-02 00:00:00,NaN,28210,North Carolina,7-USA Balance,3,258.0,NaN


In [3]:
cust_df.isnull().sum()

owner_no                       0
First Order Date               0
First Contribution Date    47854
postal_code                    0
state_desc                     0
geo_area_desc                  0
OP Prelim Capacity         21682
LTV Tkt Value                  0
Lifetime Giving            47854
dtype: int64

In [4]:
#Import groups of order data
appended_data = []
for file in glob.glob('Ord*'):
    data = pd.read_csv(file)
    appended_data.append(data)
order_df = pd.concat(appended_data)
order_df.head()

,owner_no,order_dt,Count of order_no,channel_desc,MOS_desc,delivery_desc,tot_ticket_paid_amt,tot_contribution_paid_amt,facility_desc,prod_season_desc,num_seats_pur
0,18251,2014-04-24 00:00:00,1,Phone,Ticketing,OP - US Mail,$1044,$6000,Academy of Music,Don Carlo,2
1,18251,2014-04-24 00:00:00,1,Phone,Ticketing,OP - US Mail,$1044,$6000,Academy of Music,Oscar,2
2,18251,2014-04-24 00:00:00,1,Phone,Ticketing,OP - US Mail,$1044,$6000,Academy of Music,The Barber of Seville,2
3,18251,2014-04-24 00:00:00,1,Phone,Ticketing,OP - US Mail,$1044,$6000,General Admission,40th Anniversary Voucher,2
4,18251,2014-04-24 00:00:00,1,Phone,Ticketing,OP - US Mail,$1044,$6000,Perelman,Ariadne auf Naxos,2


In [5]:
order_df.isnull().sum()

owner_no                     0
order_dt                     0
Count of order_no            0
channel_desc                 0
MOS_desc                     0
delivery_desc                0
tot_ticket_paid_amt          0
tot_contribution_paid_amt    0
facility_desc                0
prod_season_desc             0
num_seats_pur                0
dtype: int64

In [6]:
df = pd.merge(order_df, cust_df, how='inner', on = 'owner_no')
df.shape

(82407, 19)

### Handle NaN values ###

In [7]:
df.isnull().sum()

owner_no                         0
order_dt                         0
Count of order_no                0
channel_desc                     0
MOS_desc                         0
delivery_desc                    0
tot_ticket_paid_amt              0
tot_contribution_paid_amt        0
facility_desc                    0
prod_season_desc                 0
num_seats_pur                    0
First Order Date                 0
First Contribution Date      64786
postal_code                      0
state_desc                       0
geo_area_desc                    0
OP Prelim Capacity           20918
LTV Tkt Value                    0
Lifetime Giving              64786
dtype: int64

In [8]:
#Clean up OP Prelim Capacity
df['OP Prelim Capacity'] = df['OP Prelim Capacity'].fillna(0)
df['OP Prelim Capacity'] = df['OP Prelim Capacity'].replace('U',0)
df['OP Prelim Capacity'] = df['OP Prelim Capacity'].replace('X',0)

In [9]:
#Clean up First Contribution Date
df['First Contribution Date'] = df['First Contribution Date'].fillna('01-01-1900')

### Data Type Cleanup ###

In [10]:
#Create datetime data types
df.order_dt=pd.to_datetime(df.order_dt, errors='coerce')
df['First Order Date'] = pd.to_datetime(df['First Order Date'], errors='coerce')
df['First Contribution Date'] = pd.to_datetime(df['First Contribution Date'], errors='coerce')

In [11]:
df.dtypes

owner_no                              int64
order_dt                     datetime64[ns]
Count of order_no                     int64
channel_desc                         object
MOS_desc                             object
delivery_desc                        object
tot_ticket_paid_amt                  object
tot_contribution_paid_amt            object
facility_desc                        object
prod_season_desc                     object
num_seats_pur                         int64
First Order Date             datetime64[ns]
First Contribution Date      datetime64[ns]
postal_code                          object
state_desc                           object
geo_area_desc                        object
OP Prelim Capacity                   object
LTV Tkt Value                       float64
Lifetime Giving                      object
dtype: object

In [12]:
#Create numerical data types
df.tot_ticket_paid_amt = df.tot_ticket_paid_amt.str.replace('$','')
df.tot_contribution_paid_amt = df.tot_contribution_paid_amt.str.replace('$','')
df['Lifetime Giving'] = df['Lifetime Giving'].str.replace('$','')
df.tot_ticket_paid_amt = pd.to_numeric(df.tot_ticket_paid_amt, errors='coerce')
df.tot_contribution_paid_amt = pd.to_numeric(df.tot_contribution_paid_amt, errors='coerce')
df['Lifetime Giving'] = pd.to_numeric(df['Lifetime Giving'], errors='coerce')
df['OP Prelim Capacity'] = pd.to_numeric(df['OP Prelim Capacity'], errors='coerce')

### Imputing Data ###

In [13]:
#Impute total ticket paid amount
df.tot_ticket_paid_amt = df.tot_ticket_paid_amt.fillna(df.tot_ticket_paid_amt.mean())

In [14]:
#Clean up Lifetime Giving
df['Lifetime Giving'] = df['Lifetime Giving'].fillna(0)

### Additional Data Cleanup ###

In [15]:
#Drop helper column
df = df.drop('Count of order_no', axis=1)

In [16]:
#Clean up column headers
df = df.rename(
    columns={
        'First Order Date': 'first_order_dt',
        'First Contribution Date': 'first_cont_dt',
        'OP Prelim Capacity': 'prelim_capacity',
        'LTV Tkt Value': 'ltv_tkt_value',
        'OP Prelim Capacity':'Prelim Capacity'
    }
     )

### Group column values ###
Grouping our categorical variables will reduce the dataset's dimensionality and improve performance. Many of these values have a very similar role from the consumer perspective, or have replaced one another over time.

In [17]:
#Clean up channel values
channel_values = dict.fromkeys([
    'TP Box Office', 'TP Phone', 'TP - Default Channel','TodayTix','Telemarketing','FringeArts'], '3rd Party')
channel_values['Web'] = 'Online'
channel_values['Web Donations'] = 'Online'
channel_values['TP Mobile'] = 'Mobile'
df.channel_desc = df.channel_desc.replace(channel_values)

In [18]:
#Clean up delivery types
willcall_values = dict.fromkeys(
    ['Hold at Box Office', 
     'BO - Window', 
     'OP - Will Call', 
     'Box Office Window', 
     'OP - Hold at Box Office', 
     'Information Table',
     'VIP',
     'Guest Services',
     'Hand Delivered', 
     'Press'],
    'Will Call'
)
digital_values = dict.fromkeys(
    ['Email Print at Home',
     'Mobile App Ticket'], 
    'Digital')

mail_values = dict.fromkeys([
    'OP - US Mail',
    'BO - US Mail',
    'U.S. Mail',
    'Fedex-2 bus. day ($25)'],
    'Mail')
willcall_values.update(digital_values)
willcall_values.update(mail_values)
df.delivery_desc = df.delivery_desc.replace(willcall_values)

In [19]:
#Clean up MOS values
internal_values = dict.fromkeys([
    'Ticketing Web Stbs',
    'Ticketing Web Subs',
    'Web Advance Rush',
    'Web Allocation',
    'Web Allocation 2',
    'Web Allocation 3',
    'Web Allocation 4'], 
    'Internal')
external_values = dict.fromkeys([
    'TP Box Office',
    'xxTP App/Web Sales',
    'xx1 TP KC BO',
    'Curtis Subscription',
    'FringeArts',
    'TodayTix',
    'xxTP Phone Sales',
    'TP Phone',
    'TP Web OP Student Advance (NF)',
    'xxTP Web SPCl Allocation',
    'TP Phone OP Subs',
    'TP Phone Special Offer', 
    'xxTP Exchanges OP Phone'], 
    'External')
internal_values.update(external_values)
df.MOS_desc = df.MOS_desc.replace(internal_values)

In [20]:
#function to define facility values
def facilities(facility):
    if facility in ['Academy of Music', 'Independence Mall']:
        return facility
    elif facility in ['Perelman Theater','Perelman']:
        facility = 'Perelman'
        return facility
    elif facility in ['Academy of Music Ballroom',
                     'The Loft on Tier 1',
                     'Garden Restaurant',
                      'Tier 2 Lounge',
                      'Union League of Philadelphia',
                      'Estia Restaurant',
                      'Hamilton Garden',
                      'Creperie Beau Monde'
                     ]:
        facility = 'Fundraiser'
        return facility
    elif facility in ['The Barnes Foundation',
                     'Theatre of Living Arts',
                     'Suzanne Roberts',
                     'The Wilma Theater',
                     'Philadelphia Museum of Art',
                     'Prince Theater',
                     'FringeArts',
                     'Field Concert Hall',
                     'TLA']:
        facility = 'Small venue'
        return facility
    else:
        facility = 'Other'
        return facility
df.facility_desc = df.facility_desc.apply(facilities)

In [21]:
#Identify customers in the BRD, MGP, IDP, and LCM constituencies
drops_df = pd.read_excel('drops.xlsx')
prosp_brd_values = list(drops_df['customer_no'])
df['prospect_board'] = df.owner_no.apply(lambda x: 1 if x in prosp_brd_values else 0)

In [22]:
#Function to create calculated field for difference between first order and first contribution
def days_to_donation(order_dt, cont_dt):
    if cont_dt == datetime.date(1900,1,1):
        result = -100
    elif cont_dt < order_dt:
        result = -1
    else:
        result = (cont_dt - order_dt).days
    return result

df['days_to_donation'] = df.apply(lambda x: days_to_donation(x.first_order_dt, x.first_cont_dt), axis=1)

In [26]:
df['first_cont_order'] = np.where(df.first_cont_dt == df.first_order_dt, 1, 0)

In [27]:
df['first_cont_after'] = np.where(df.first_cont_dt > df.first_order_dt, 1, 0)

### Drop Categorical Variables Summarized by Other Variables ####
We can drop a few more variables that are summarized by other dimensions.

In [28]:
df = df.drop(['postal_code','state_desc','prod_season_desc'], axis=1)

In [29]:
#Create categorical data types
df.channel_desc = df.channel_desc.astype('category')
df.MOS_desc = df.MOS_desc.astype('category')
df.delivery_desc = df.delivery_desc.astype('category')
df.facility_desc = df.facility_desc.astype('category')
df.geo_area_desc = df.geo_area_desc.str.split('-').str[0]
df.geo_area_desc = df.geo_area_desc.astype('int')

In [30]:
df.dtypes

owner_no                              int64
order_dt                     datetime64[ns]
channel_desc                       category
MOS_desc                           category
delivery_desc                      category
tot_ticket_paid_amt                 float64
tot_contribution_paid_amt           float64
facility_desc                      category
num_seats_pur                         int64
first_order_dt               datetime64[ns]
first_cont_dt                datetime64[ns]
geo_area_desc                         int32
Prelim Capacity                       int64
ltv_tkt_value                       float64
Lifetime Giving                     float64
prospect_board                        int64
days_to_donation                      int64
first_cont_order                      int32
first_cont_after                      int32
dtype: object

Lastly, let's export the data to csv

In [31]:
df = pd.get_dummies(df)
df.columns

Index(['owner_no', 'order_dt', 'tot_ticket_paid_amt',
       'tot_contribution_paid_amt', 'num_seats_pur', 'first_order_dt',
       'first_cont_dt', 'geo_area_desc', 'Prelim Capacity', 'ltv_tkt_value',
       'Lifetime Giving', 'prospect_board', 'days_to_donation',
       'first_cont_order', 'first_cont_after', 'channel_desc_3rd Party',
       'channel_desc_At the Performance', 'channel_desc_Chat',
       'channel_desc_Default Channel', 'channel_desc_Email',
       'channel_desc_Fax', 'channel_desc_Internal Request',
       'channel_desc_Mail', 'channel_desc_Mobile', 'channel_desc_Online',
       'channel_desc_Phone', 'channel_desc_Telefunding',
       'channel_desc_Walk Up', 'MOS_desc_External', 'MOS_desc_Internal',
       'MOS_desc_Ticketing', 'delivery_desc_Digital',
       'delivery_desc_Do Not Print Tickets', 'delivery_desc_Mail',
       'delivery_desc_Will Call', 'facility_desc_Academy of Music',
       'facility_desc_Fundraiser', 'facility_desc_Independence Mall',
       'facili

In [32]:
cols = list(df.columns)
cols.remove('num_seats_pur')
sum_dict = {'num_seats_pur':'sum'}
agg_dict = dict.fromkeys(cols,'max')

In [33]:
df = df.groupby(['owner_no','order_dt']).agg(agg_dict)

In [34]:
pd.set_option('display.max_columns',50)
df.reset_index(drop=True)

,owner_no,order_dt,tot_ticket_paid_amt,tot_contribution_paid_amt,first_order_dt,first_cont_dt,geo_area_desc,Prelim Capacity,ltv_tkt_value,Lifetime Giving,prospect_board,days_to_donation,first_cont_order,first_cont_after,channel_desc_3rd Party,channel_desc_At the Performance,channel_desc_Chat,channel_desc_Default Channel,channel_desc_Email,channel_desc_Fax,channel_desc_Internal Request,channel_desc_Mail,channel_desc_Mobile,channel_desc_Online,channel_desc_Phone,channel_desc_Telefunding,channel_desc_Walk Up,MOS_desc_External,MOS_desc_Internal,MOS_desc_Ticketing,delivery_desc_Digital,delivery_desc_Do Not Print Tickets,delivery_desc_Mail,delivery_desc_Will Call,facility_desc_Academy of Music,facility_desc_Fundraiser,facility_desc_Independence Mall,facility_desc_Other,facility_desc_Perelman,facility_desc_Small venue
0,111,2015-10-07,138.0,0.0,2015-10-07,1900-01-01,1,5,366.0,0.0,0,-100,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0
1,111,2016-09-16,158.0,0.0,2015-10-07,1900-01-01,1,5,366.0,0.0,0,-100,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0
2,111,2018-09-21,70.0,0.0,2015-10-07,1900-01-01,1,5,366.0,0.0,0,-100,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0
3,258,2014-09-28,169.0,0.0,2014-09-28,1900-01-01,5,6,169.0,0.0,0,-100,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0
4,262,2014-10-02,45.0,0.0,2014-10-02,1900-01-01,2,4,90.0,0.0,0,-100,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70367,2419011,2018-09-07,0.0,25.0,2018-09-07,2018-09-07,2,5,0.0,50.0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0
70368,2419126,2019-09-19,0.0,0.0,2019-09-19,1900-01-01,1,0,600.0,0.0,0,-100,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0
70369,2419248,2017-09-20,440.0,0.0,2017-09-20,2019-12-11,2,4,440.0,100.0,0,812,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0
70370,2419278,2015-04-13,395.0,0.0,2011-04-03,2003-03-26,1,3,752.0,75.0,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0


In [37]:
df.to_csv('1.1 Processed Data.csv')